In [ ]:
#working using yolo v3/v4

import numpy as np
import cv2
from scipy.optimize import linear_sum_assignment

class CentroidTracker:
    def __init__(self, max_disappeared=50):
        self.next_object_id = 1
        self.objects = {}
        self.disappeared = {}
        self.max_disappeared = max_disappeared
        self.used_ids = set()

    def register(self, centroid):
        object_id = self.next_object_id
        self.objects[object_id] = centroid
        self.disappeared[object_id] = 0
        self.used_ids.add(object_id)
        self.next_object_id += 1
        return object_id

    def deregister(self, object_id):
        del self.objects[object_id]
        del self.disappeared[object_id]
        self.used_ids.remove(object_id)

    def update(self, centroids):
        if len(centroids) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        input_centroids = np.array(centroids)

        if len(self.objects) == 0:
            for i in range(0, len(centroids)):
                object_id = self.register(centroids[i])
        else:
            object_ids = list(self.objects.keys())
            object_centroids = np.array(list(self.objects.values()))

            D = np.zeros((len(object_ids), len(centroids)))
            for i, object_centroid in enumerate(object_centroids):
                for j, centroid in enumerate(input_centroids):
                    D[i, j] = np.linalg.norm(object_centroid - centroid)

            row_ind, col_ind = linear_sum_assignment(D)

            used_rows = set()
            used_cols = set()

            for row, col in zip(row_ind, col_ind):
                if row in used_rows or col in used_cols:
                    continue

                object_id = object_ids[row]
                self.objects[object_id] = centroids[col]
                self.disappeared[object_id] = 0

                used_rows.add(row)
                used_cols.add(col)

            unused_rows = set(range(len(object_ids))).difference(used_rows)
            unused_cols = set(range(len(centroids))).difference(used_cols)

            for row in unused_rows:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1

                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)

            for col in unused_cols:
                self.register(centroids[col])

        return self.objects



def detect_persons(frame, net, output_layers):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2 and class_id == 0:  # Class ID 0 corresponds to 'person' in COCO dataset
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w // 2
                y = center_y - h // 2
                boxes.append([x, y, x+w, y+h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    centroids = []
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            centroid = ((x + w) // 2, (y + h) // 2)
            centroids.append(centroid)

    return centroids

def draw_boxes(img, objects, roi, counts):
    for object_id, centroid in objects.items():
        cv2.circle(img, centroid, 4, (0, 255, 0), -1)
        cv2.putText(img, f'ID: {object_id}', (centroid[0] - 10, centroid[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Check if the centroid is within the ROI
        x, y, w, h = roi
        if x < centroid[0] < x + w and y < centroid[1] < y + h:
            
            if object_id not in counts:
                counts[object_id] = 0  # Initialize count for new object
            counts[object_id] += 1
            
            cv2.putText(img, 'Passed ROI', (centroid[0] - 10, centroid[1] + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    # Draw ROI rectangle
    cv2.rectangle(img, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (255, 0, 0), 2)

    # Display count for each object
    #for object_id, count in counts.items():
     #   cv2.putText(img, f'Count {object_id}: {count}', (10, 30 * (object_id + 1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(img, f'Count: {len(counts)}', (10, 30 ), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return img

def main():
    tracker = CentroidTracker()

    # Load YOLO
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    cap = cv2.VideoCapture(r"C:\Users\Chandhana\Downloads\Test video half.mp4")  # Path to your sample video

    # Define the ROI coordinates (x, y, width, height)
    roi = (0, 200, 1209, 10)
    # Initialize count variable
    counts = {}  # Dictionary to store count for each object
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        centroids = detect_persons(frame, net, output_layers)

        tracked_objects = tracker.update(centroids)

        frame = draw_boxes(frame, tracked_objects, roi, counts)

        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    print("Total persons crossed ROI:", len(counts))

if __name__ == "__main__":
    main()
